In [25]:
from typing import Literal
from tqdm import tqdm
from langchain.agents import initialize_agent, load_tools, AgentType
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_experimental.plan_and_execute import (
    load_chat_planner, load_agent_executor, PlanAndExecute
)
from sentence_transformers import SentenceTransformer

import pandas as pd

/HDD1/bbq9088/miniconda3/envs/dacon/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# load CSV
csv_name = "train_data_smi_unique_target_name"
csv_dir = "./target_csv/"
df = pd.read_csv(csv_dir + csv_name +".csv")

In [26]:
# 임베딩 모델 로드
embedding_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/HDD1/bbq9088/miniconda3/envs/dacon/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
ReasoningStrategies = Literal["zero-shot-react", "plan-and-solve"]

def load_agent(
    tool_names: list[str],
    strategy: ReasoningStrategies = "zero-shot-react"
) -> LLMChain:
    llm = ChatOllama(model="llama3.1:70b", temperature=0, streaming=True)
    tools = load_tools(
        tool_names=tool_names,
        llm=llm
    )
    
    if strategy == "plan-and-solve":
        planner = load_chat_planner(llm)
        executor = load_agent_executor(llm, tools, verbose=True)
        return PlanAndExecute(planner=planner, executor=executor, verbose=True)

    return initialize_agent(
        tools=tools, llm=llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True
    )

In [4]:
# 에이전트 로드
tool_names = ["wikipedia", ""]
agent_chain = load_agent(tool_names, strategy="plan-and-solve")

In [5]:
# Description 컬럼을 생성하여 각 protein_name에 대한 설명을 추가 및 저장
if "description" not in df.columns:
    df["description"] = ""

In [6]:
save_threshold = 10  # 열 개마다 저장
save_count = 0

In [7]:
for idx, protein_name in tqdm(enumerate(df["Target Name"]), desc="Fetching descriptions", total=len(df)):
    if not df.at[idx, "description"]:  # 이미 설명이 채워진 경우 건너뜀
        query = f"Please provide a brief description of {protein_name}."
        try:
            description = agent_chain.invoke(query, handle_parsing_errors=True)
        except ValueError as e:
            description = f"Error: {str(e)}"
        
        df.at[idx, "description"] = description
        save_count += 1

        # 열 개의 설명이 추가되면 저장
        if save_count >= save_threshold:
            df.to_csv(csv_name + " cnt," + str(save_count) + ".csv", index=False)
            save_count = 0  # 저장 후 카운트 초기화

Fetching descriptions:   0%|                                                                                                                                                                                                                                   | 0/61 [00:00<?, ?it/s]



> Entering new PlanAndExecute chain...
steps=[Step(value='Research and gather information on Tyrosine-protein kinase receptor RET from credible sources such as scientific journals, databases, or reputable websites.'), Step(value='Identify the key characteristics, functions, and roles of RET in cellular processes and human health.'), Step(value="Organize the gathered information into a concise and accurate description.\n\nGiven the above steps taken, please respond to the user's original question:\n\nTyrosine-protein kinase receptor RET (RET) is a proto-oncogene that encodes a receptor tyrosine kinase for glial cell line-derived neurotrophic factor (GDNF) family of extracellular signaling molecules. It plays a crucial role in the development and maintenance of the nervous system, particularly in the formation and survival of neurons. RET is also involved in the regulation of cell growth, differentiation, and migration.\n\nRET mutations have been associated with multiple endocrine neop

/HDD1/bbq9088/miniconda3/envs/dacon/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /HDD1/bbq9088/miniconda3/envs/dacon/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: RET proto-oncogene
Summary: The RET proto-oncogene encodes a receptor tyrosine kinase for members of the glial cell line-derived neurotrophic factor (GDNF) family of extracellular signalling molecules.
RET loss of function mutations are associated with the development of Hirschsprung's disease, while gain of function mutations are associated with the development of various types of human cancer, including medullary thyroid carcinoma, multiple endocrine neoplasias type 2A and 2B, pheochromocytoma and parathyroid hyperplasia.

Page: Multiple endocrine neoplasia type 2
Summary: Multiple endocrine neoplasia type 2 (also known as "Pheochromocytoma (codons 630 and 634) and amyloid producing medullary thyroid carcinoma", "PTC syndrome," and "Sipple syndrome") is a group of medical disorders associated with tumors of the endocrine system. The tumors may be benign or malignant (cancer).  They generally occur in endocrine organs (e.g. thyroid, parathyroid, and adrenals), but 

/HDD1/bbq9088/miniconda3/envs/dacon/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /HDD1/bbq9088/miniconda3/envs/dacon/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


...

Based on the provided steps, I will now respond to the user's original question.

The user asked about stem cells and their role in the body. Stem cells are essential for the development, growth, and maintenance of the body. They have the ability to self-renew and differentiate into different cell types, tissues, or organs. Stem cells play a crucial role in regenerating damaged tissues, maintaining tissue homeostasis, and supporting immune function.

Growth factors, such as stem cell factor (SCF), interact with specific receptors on the surface of cells, including stem cells, to regulate various cellular processes. The stem cell growth factor receptor, c-KIT, is a transmembrane tyrosine kinase receptor that plays a crucial role in the development, survival, and function of hematopoietic cells, including stem cells.

In summary, stem cells are essential for the body's development, growth, and maintenance, and their dysregulation has been implicated in various diseases. Growth factors an

Fetching descriptions:  98%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍   | 60/61 [47:40<00:47, 47.31s/it]

Serine/threonine-protein kinase AKT3 is a member of the AKT family of serine/threonine kinases that plays a crucial role in cell survival, growth, and metabolism. It phosphorylates various substrates involved in these processes and is implicated in the PI3K/AKT signaling pathway. AKT3 is expressed in various tissues, including brain, heart, and skeletal muscle, and localizes to the cytoplasm and nucleus.

> Finished chain.
*****

Step: Summarize key points about Serine/threonine-protein kinase AKT3.

Given the above steps taken, please respond to the user's original question:

Serine/threonine-protein kinase AKT3 is a member of the AKT family of serine/threonine kinases that plays a crucial role in cell survival, growth, and metabolism. It phosphorylates various substrates involved in these processes and is implicated in the PI3K/AKT signaling pathway. AKT3 is expressed in various tissues, including brain, heart, and skeletal muscle, and localizes to the cytoplasm and nucleus.



Respo

Fetching descriptions: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [48:09<00:00, 47.37s/it]

The PI3-kinase p110-gamma subunit is an isoform of the phosphoinositide 3-kinase (PI3K) enzyme that plays a specific role in cell signaling pathways, particularly in regulating inflammation and immune responses. It forms a heterodimer with regulatory subunits to form the active PI3K enzyme complex.

> Finished chain.
*****

Step: Gather information on its structure and interactions: The p110-gamma subunit forms a heterodimer with a regulatory subunit, typically p101 or p84/87, to form the active PI3K enzyme complex.

Given the above steps taken, please respond to the user's original question:

The PI3-kinase p110-gamma subunit is an isoform of the phosphoinositide 3-kinase (PI3K) enzyme that plays a specific role in cell signaling pathways, particularly in regulating inflammation and immune responses. It forms a heterodimer with regulatory subunits to form the active PI3K enzyme complex.



Response: The PI3-kinase p110-gamma subunit is an isoform of the phosphoinositide 3-kinase (PI3K

In [8]:
# 모든 작업이 끝난 후 마지막으로 저장 (남아있는 데이터 저장)
if save_count > 0:
    df.to_csv(csv_name + "_descriptions.csv", index=False)

In [29]:
# JSon, output만 분리
df['description'].apply(lambda x: x['output'])

TypeError: string indices must be integers, not 'str'

In [31]:
# 임베딩을 Target Name에 대응시키는 딕셔너리 생성
df['embedding'] = df['description'].apply(
    lambda text: None if text is None else embedding_model.encode(text)
)

In [32]:
df.to_csv(csv_name + "_descriptions.csv", index=False)

In [9]:
# # 예시 도구 이름 리스트
# tool_names = ["wikipedia"]

# # 에이전트 로드 (제로샷 모델 사용)
# agent_chain = load_agent(tool_names, strategy="zero-shot-react")

# # 에이전트 실행
# result = agent_chain.run("Explain about T1-Faker")

# print("Result:\n", result)